# Running Tune experiments with Optuna

In this tutorial we introduce Optuna, while running a simple Ray Tune experiment. Tune’s Search Algorithms integrate with Optuna and, as a result, allow you to seamlessly scale up a Optuna optimization process - without sacrificing performance.

Similar to Ray Tune, Optuna is an automatic hyperparameter optimization software framework, particularly designed for machine learning. It features an imperative ("how" over "what" emphasis), define-by-run style user API. With Optuna, a user has the ability to dynamically construct the search spaces for the hyperparameters. Optuna falls in the domain of "derivative-free optimization" and "black-box optimization".

In this example we minimize a simple objective to briefly demonstrate the usage of Optuna with Ray Tune via `OptunaSearch`, including examples of conditional search spaces (string together relationships between hyperparameters), and the multi-objective problem (measure trade-offs among all important metrics). It's useful to keep in mind that despite the emphasis on machine learning experiments, Ray Tune optimizes any implicit or explicit objective. Here we assume `optuna==2.9.1` library is installed. To learn more, please refer to [Optuna website](https://optuna.org/).

Please note that sophisticated schedulers, such as `AsyncHyperBandScheduler`, may not work correctly with multi-objective optimization, since they typically expect a scalar score to compare fitness among trials.


In [1]:
# !pip install ray[tune]
!pip install optuna==2.9.1

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

In [2]:
import time
from typing import Dict, Optional, Any

import ray
from ray import tune
from ray.air import session
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.optuna import OptunaSearch

In [3]:
ray.init(configure_logging=False)

Python version:,3.7.7
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8265


Let's start by defining a simple evaluation function.
An explicit math formula is queried here for demonstration, yet in practice this is typically a black-box function-- e.g. the performance results after training an ML model.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment while tuning three hyperparameters,
namely `width`, `height`, and `activation`.

In [4]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our `objective` function to be optimized takes a Tune `config`, evaluates the `score` of your experiment in a training loop,
and uses `session.report` to report the `score` back to Tune.

In [5]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        session.report({"iterations": step, "mean_loss": score})
                  

Next we define a search space. The critical assumption is that the optimal hyperparamters live within this space. Yet, if the space is very large, then those hyperparamters may be difficult to find in a short amount of time.

The simplest case is a search space with independent dimensions. In this case, a config dictionary will suffice.

In [6]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"]),
}

Here we define the Optuna search algorithm:

In [7]:
algo = OptunaSearch()

We also constrain the the number of concurrent trials to `4` with a `ConcurrencyLimiter`.

In [8]:
algo = ConcurrencyLimiter(algo, max_concurrent=4)


The number of samples is the number of hyperparameter combinations that will be tried out. This Tune run is set to `1000` samples.
(you can decrease this if it takes too long on your machine).

In [9]:
num_samples = 1000

In [10]:
# We override here for our smoke tests.
num_samples = 10

Finally, we run the experiment to `"min"`imize the "mean_loss" of the `objective` by searching `search_space` via `algo`, `num_samples` times. This previous sentence is fully characterizes the search problem we aim to solve. With this in mind, notice how efficient it is to execute `tuner.fit()`.

In [11]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_space,
)
results = tuner.fit()

Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
[I 2022-07-22 15:21:47,769] A new study created in memory with name: optuna


Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_9f689668,TERMINATED,127.0.0.1:46141,tanh,32.9806,16.0406,3.36064,100,11.0409,99,-3.36064
objective_a11d2104,TERMINATED,127.0.0.1:46149,relu,72.2627,0.763881,18.3942,100,11.7018,99,-18.3942
objective_a11eef8e,TERMINATED,127.0.0.1:46150,relu,-32.8474,13.1144,6.79169,100,11.8071,99,-6.79169
objective_a1209e56,TERMINATED,127.0.0.1:46151,tanh,75.6408,10.4415,7.65989,100,11.6987,99,-7.65989
objective_a7b383e6,TERMINATED,127.0.0.1:46174,relu,50.1501,3.4612,15.2986,100,10.7015,99,-15.2986
objective_a9c844c8,TERMINATED,127.0.0.1:46183,relu,-40.3931,14.0525,6.03205,100,10.6687,99,-6.03205
objective_a9cb308e,TERMINATED,127.0.0.1:46184,tanh,48.4802,12.7746,4.92647,100,10.7122,99,-4.92647
objective_a9d8332e,TERMINATED,127.0.0.1:46189,tanh,-1.26682,10.9788,-0.0355157,100,10.7089,99,0.0355157
objective_af948a1a,TERMINATED,127.0.0.1:46204,tanh,61.6381,11.0105,6.25472,100,10.744,99,-6.25472
objective_b1bad2fe,TERMINATED,127.0.0.1:46209,tanh,-16.4629,7.68847,-1.51661,100,10.7274,99,1.51661


Result for objective_9f689668:
  date: 2022-07-22_15-21-51
  done: false
  experiment_id: 7d29e5c95aa44d77becc24b23a326b9b
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 13.298063134872326
  neg_mean_loss: -13.298063134872326
  node_ip: 127.0.0.1
  pid: 46141
  time_since_restore: 0.10468196868896484
  time_this_iter_s: 0.10468196868896484
  time_total_s: 0.10468196868896484
  timestamp: 1658499711
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 9f689668
  warmup_time: 0.003930091857910156
  
Result for objective_a1209e56:
  date: 2022-07-22_15-21-54
  done: false
  experiment_id: 70d9788c327c43fb94450e5084416b81
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 17.564083106485064
  neg_mean_loss: -17.564083106485064
  node_ip: 127.0.0.1
  pid: 46151
  time_since_restore: 0.10416483879089355
  time_this_iter_s: 0.10416483879089355
  time_total_s: 0.10416483879089355
  timestamp: 

And now we have the hyperparameters found to minimize the mean loss.

In [12]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 7.688465886501621, 'height': -16.46289560854555, 'activation': 'tanh'}


## Providing an initial set of hyperparameters

While defining the search algorithm, we may choose to provide an initial set of hyperparameters that we believe are especially promising or informative, and
pass this information as a helpful starting point for the `OptunaSearch` object.

In [13]:
initial_params = [
    {"width": 1, "height": 2, "activation": "relu"},
    {"width": 4, "height": 2, "activation": "relu"},
]

Now the `search_alg` built using `OptunaSearch` takes `points_to_evaluate`.

In [14]:
searcher = OptunaSearch(points_to_evaluate=initial_params)
algo = ConcurrencyLimiter(searcher, max_concurrent=4)

And run the experiment with initial hyperparameter evaluations:

In [15]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_loss",
        mode="min",
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_space,
)
results = tuner.fit()

[I 2022-07-22 15:22:32,644] A new study created in memory with name: optuna
/Users/kai/coding/ray/python/ray/tune/search/optuna/optuna_search.py:389: ExperimentalWarning: enqueue_trial is experimental (supported from v1.2.0). The interface can change in the future.
  self._ot_study.enqueue_trial(point)
/Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/optuna/study/study.py:857: ExperimentalWarning: create_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})
/Users/kai/.pyenv/versions/3.7.7/lib/python3.7/site-packages/optuna/study/study.py:857: ExperimentalWarning: add_trial is experimental (supported from v2.0.0). The interface can change in the future.
  create_trial(state=TrialState.WAITING, system_attrs={"fixed_params": params})


Trial name,status,loc,activation,height,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_b9acff32,TERMINATED,127.0.0.1:46220,relu,2,1,11.1174,100,10.6837,99,-11.1174
objective_bb3b2702,TERMINATED,127.0.0.1:46227,relu,2,4,10.4463,100,11.2816,99,-10.4463
objective_bb3c8714,TERMINATED,127.0.0.1:46228,relu,-90.7016,0.566597,2.44285,100,11.2766,99,-2.44285
objective_bb3e0210,TERMINATED,127.0.0.1:46229,tanh,-81.4501,14.783,-8.07715,100,11.2196,99,8.07715
objective_c19b9dde,TERMINATED,127.0.0.1:46248,relu,86.466,0.308659,21.1122,100,10.7226,99,-21.1122
objective_c3762426,TERMINATED,127.0.0.1:46253,tanh,-98.4632,0.723671,-8.62148,100,10.7271,99,8.62148
objective_c385528e,TERMINATED,127.0.0.1:46256,relu,-54.8368,11.0765,4.60669,100,10.7498,99,-4.60669
objective_c387a7c8,TERMINATED,127.0.0.1:46257,tanh,42.9466,8.01708,4.41909,100,10.7214,99,-4.41909
objective_c97678f8,TERMINATED,127.0.0.1:46272,tanh,9.25214,15.5288,0.989841,100,12.4591,99,-0.989841
objective_cb6180fe,TERMINATED,127.0.0.1:46277,relu,41.6647,8.0585,14.2903,100,10.752,99,-14.2903


Result for objective_b9acff32:
  date: 2022-07-22_15-22-35
  done: false
  experiment_id: ec3c260f0ad348fa9b188f9c93c01eba
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 20.2
  neg_mean_loss: -20.2
  node_ip: 127.0.0.1
  pid: 46220
  time_since_restore: 0.10049605369567871
  time_this_iter_s: 0.10049605369567871
  time_total_s: 0.10049605369567871
  timestamp: 1658499755
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: b9acff32
  warmup_time: 0.0029239654541015625
  
Result for objective_bb3e0210:
  date: 2022-07-22_15-22-37
  done: false
  experiment_id: 08a7bc47c140405296e58a049cd2f6b4
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 1.8549886153795079
  neg_mean_loss: -1.8549886153795079
  node_ip: 127.0.0.1
  pid: 46229
  time_since_restore: 0.10361814498901367
  time_this_iter_s: 0.10361814498901367
  time_total_s: 0.10361814498901367
  timestamp: 1658499757
  timesteps_sinc

We take another look at the optimal hyperparamters.

In [16]:
print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'steps': 100, 'width': 0.7236705053153614, 'height': -98.46320622400306, 'activation': 'tanh'}


## Conditional search spaces 

Sometimes we may want to build a more complicated search space that has conditional dependencies on other hyperparameters. In this case, we pass a define-by-run function to the `search_alg` argument in `ray.tune()`.

In [17]:
def define_by_run_func(trial) -> Optional[Dict[str, Any]]:
    """Define-by-run function to create the search space.

    Ensure no actual computation takes place here. That should go into
    the trainable passed to ``Tuner()`` (in this example, that's
    ``objective``).

    For more information, see https://optuna.readthedocs.io/en/stable\
    /tutorial/10_key_features/002_configurations.html

    This function should either return None or a dict with constant values.
    """

    activation = trial.suggest_categorical("activation", ["relu", "tanh"])

    # Define-by-run allows for conditional search spaces.
    if activation == "relu":
        trial.suggest_float("width", 0, 20)
        trial.suggest_float("height", -100, 100)
    else:
        trial.suggest_float("width", -1, 21)
        trial.suggest_float("height", -101, 101)
        
    # Return all constants in a dictionary.
    return {"steps": 100}

As before, we create the `search_alg` from `OptunaSearch` and `ConcurrencyLimiter`, this time we define the scope of search via the `space` argument and provide no initialization. We also must specific metric and mode when using `space`. 

In [18]:
searcher = OptunaSearch(space=define_by_run_func, metric="mean_loss", mode="min")
algo = ConcurrencyLimiter(searcher, max_concurrent=4)

[I 2022-07-22 15:23:15,784] A new study created in memory with name: optuna


Running the experiment with a define-by-run search space:

In [19]:
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        search_alg=algo,
        num_samples=num_samples,
    ),
)
results = tuner.fit()

Trial name,status,loc,activation,height,steps,width,loss,iter,total time (s),iterations,neg_mean_loss
objective_d363eed6,TERMINATED,127.0.0.1:46290,relu,96.9804,100,6.47293,19.8517,100,10.735,99,-19.8517
objective_d4f3a700,TERMINATED,127.0.0.1:46298,tanh,-76.4387,100,17.7358,-7.58724,100,11.396,99,7.58724
objective_d4f50ce4,TERMINATED,127.0.0.1:46299,relu,-89.8609,100,6.28321,1.17212,100,11.4182,99,-1.17212
objective_d4f67908,TERMINATED,127.0.0.1:46300,relu,67.9266,100,14.0599,16.864,100,11.341,99,-16.864
objective_db5c0402,TERMINATED,127.0.0.1:46317,tanh,-54.13,100,11.4712,-5.32571,100,10.705,99,5.32571
objective_dd487b56,TERMINATED,127.0.0.1:46322,relu,-25.9818,100,0.758497,8.57703,100,10.6916,99,-8.57703
objective_dd4b4e94,TERMINATED,127.0.0.1:46323,relu,54.6085,100,1.2361,16.2163,100,10.6711,99,-16.2163
objective_dd5a8bca,TERMINATED,127.0.0.1:46328,tanh,5.22131,100,17.0952,0.580871,100,10.7017,99,-0.580871
objective_e3482178,TERMINATED,127.0.0.1:46341,tanh,78.844,100,15.5079,7.94912,100,12.6417,99,-7.94912
objective_e532d6e0,TERMINATED,127.0.0.1:46353,tanh,-66.9988,100,7.15087,-6.56059,100,10.7486,99,6.56059


Result for objective_d363eed6:
  date: 2022-07-22_15-23-18
  done: false
  experiment_id: af8f846e60254bc794827e76909df4f0
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 29.698044266420816
  neg_mean_loss: -29.698044266420816
  node_ip: 127.0.0.1
  pid: 46290
  time_since_restore: 0.10497498512268066
  time_this_iter_s: 0.10497498512268066
  time_total_s: 0.10497498512268066
  timestamp: 1658499798
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: d363eed6
  warmup_time: 0.0027971267700195312
  
Result for objective_d4f3a700:
  date: 2022-07-22_15-23-21
  done: false
  experiment_id: 52e5f2a557cf4466939b6c80bb9ff905
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  mean_loss: 2.3561338711040003
  neg_mean_loss: -2.3561338711040003
  node_ip: 127.0.0.1
  pid: 46298
  time_since_restore: 0.10145115852355957
  time_this_iter_s: 0.10145115852355957
  time_total_s: 0.10145115852355957
  timestamp:

We take a look again at the optimal hyperparameters.

In [21]:
print("Best hyperparameters for loss found were: ", results.get_best_result("mean_loss", "min").config)

Best hyperparameters for loss found were:  {'activation': 'tanh', 'width': 17.73584230792165, 'height': -76.43866128895999, 'steps': 100}


## Multi-objective optimization

Finally, let's take a look at the multi-objective case.

In [22]:
def multi_objective(config):
    # Hyperparameters
    width, height = config["width"], config["height"]

    for step in range(config["steps"]):
        # Iterative training function - can be any arbitrary training procedure
        intermediate_score = evaluate(step, config["width"], config["height"], config["activation"])
        # Feed the score back back to Tune.
        session.report({
           "iterations": step, "loss": intermediate_score, "gain": intermediate_score * width
        })

We define the `OptunaSearch` object this time with metric and mode as list arguments.

In [23]:
searcher = OptunaSearch(metric=["loss", "gain"], mode=["min", "max"])
algo = ConcurrencyLimiter(searcher, max_concurrent=4)

tuner = tune.Tuner(
    multi_objective,
    tune_config=tune.TuneConfig(
        search_alg=algo,
        num_samples=num_samples,
    ),
    param_space=search_space
)
results = tuner.fit()

[I 2022-07-22 15:26:50,680] A new study created in memory with name: optuna


Trial name,status,loc,activation,height,width,iter,total time (s),iterations,loss,gain
multi_objective_5378f1e8,TERMINATED,127.0.0.1:46621,tanh,-47.5236,10.9256,100,10.7064,99,-4.66076,-50.9216
multi_objective_550962f4,TERMINATED,127.0.0.1:46631,relu,-40.6828,4.34953,100,11.3948,99,6.15869,26.7874
multi_objective_550ad9a4,TERMINATED,127.0.0.1:46632,tanh,-42.0563,2.02124,100,11.2755,99,-3.72967,-7.53857
multi_objective_550c4faa,TERMINATED,127.0.0.1:46633,relu,-56.2838,5.65237,100,11.407,99,4.54719,25.7024
multi_objective_5b6c4ba2,TERMINATED,127.0.0.1:46679,relu,54.9133,16.9039,100,10.6698,99,15.5507,262.867
multi_objective_5d51f2d2,TERMINATED,127.0.0.1:46686,tanh,16.5793,5.16709,100,10.7451,99,1.84967,9.55744
multi_objective_5d636760,TERMINATED,127.0.0.1:46689,tanh,-13.8895,13.6043,100,10.6897,99,-1.31525,-17.8931
multi_objective_5d66093e,TERMINATED,127.0.0.1:46690,relu,-51.6714,19.8211,100,10.7246,99,4.88357,96.7974
multi_objective_634c9444,TERMINATED,127.0.0.1:46710,relu,-12.3384,15.1097,100,13.1986,99,8.83256,133.457
multi_objective_65469a2e,TERMINATED,127.0.0.1:46719,tanh,19.6774,13.6172,100,10.7279,99,2.04137,27.7978


Result for multi_objective_5378f1e8:
  date: 2022-07-22_15-26-53
  done: false
  experiment_id: 4ce03cab420a449a9f4827c3b6b76c38
  gain: 57.33359096413655
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  loss: 5.247638036988612
  node_ip: 127.0.0.1
  pid: 46621
  time_since_restore: 0.10418081283569336
  time_this_iter_s: 0.10418081283569336
  time_total_s: 0.10418081283569336
  timestamp: 1658500013
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 5378f1e8
  warmup_time: 0.002785921096801758
  
Result for multi_objective_550ad9a4:
  date: 2022-07-22_15-26-56
  done: false
  experiment_id: 102a4b06449b428b9870a75d02812be6
  gain: 11.711822825618256
  hostname: Kais-MacBook-Pro.local
  iterations: 0
  iterations_since_restore: 1
  loss: 5.794371166191576
  node_ip: 127.0.0.1
  pid: 46632
  time_since_restore: 0.10288000106811523
  time_this_iter_s: 0.10288000106811523
  time_total_s: 0.10288000106811523
  timestamp: 1658500016
  timestep

Now there are two hyperparameter sets for the two objectives.

In [24]:
print("Best hyperparameters for loss found were: ", results.get_best_result("loss", "min").config)
print("Best hyperparameters for gain found were: ", results.get_best_result("gain", "max").config)

Best hyperparameters for loss found were:  {'steps': 100, 'width': 10.925599395387751, 'height': -47.52361963011387, 'activation': 'tanh'}
Best hyperparameters for gain found were:  {'steps': 100, 'width': 16.90386360893735, 'height': 54.91329340230965, 'activation': 'relu'}


We can mix-and-match the use of initial hyperparameter evaluations, conditional search spaces via define-by-run functions, and multi-objective tasks. This is also true of scheduler usage, with the exception of multi-objective optimization-- schedulers typically rely on a single scalar score, rather than the two scores we use here: loss, gain.

In [25]:
ray.shutdown()